In [418]:
import pandas as pd

In [419]:
df = pd.read_csv('train.csv')

#### We need to add a target column which is 1 if at least 1 of the 6 columns is 1 otherwise set to 0

In [421]:
df['spam'] = ((df['toxic'] + df['severe_toxic'] + df['obscene'] + df['threat'] + df['insult'] + df['identity_hate']) >= 1).astype(int)

In [422]:
df.drop(columns=['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], inplace=True)

In [425]:
from sklearn.model_selection import train_test_split

In [426]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=1)

In [434]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [558]:
t = Tokenizer(num_words = 20000)
t.fit_on_texts(list(train_df['comment_text']))

In [559]:
def preprocessInputText(l, mlen):
    # l is a list of strings
    trans = t.texts_to_sequences(l)
    trans_padded = pad_sequences(trans, maxlen=mlen)
    return trans_padded

In [521]:
X_train = preprocessInputText(train_df['comment_text'], 100)
X_test = preprocessInputText(test_df['comment_text'],)

In [448]:
import tensorflow as tf
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [449]:
inp = Input(shape=(100, ))

In [450]:
embed_size = 128
x = Embedding(20000, embed_size)(inp)

In [451]:
x = LSTM(60, return_sequences=True, name='lstm_layer')(x)

In [452]:
x = GlobalMaxPool1D()(x)

In [453]:
x = Dropout(0.1)(x)

In [454]:
x = Dense(50, activation='relu')(x)

In [455]:
x = Dropout(0.1)(x)

In [456]:
x = Dense(1, activation='sigmoid')(x)

In [457]:
model = Model(inputs=inp, outputs=x)

In [458]:
model.compile(loss='binary_crossentropy',
optimizer='adam', metrics=['accuracy'])

In [459]:
batch_size = 32
epochs = 2
model.fit(X_train, train_df['spam'], batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/2
3591/3591 [==============================] - 333s 92ms/step - loss: 0.1404 - accuracy: 0.9514 - val_loss: 0.1065 - val_accuracy: 0.9616
Epoch 2/2
3591/3591 [==============================] - 321s 89ms/step - loss: 0.0891 - accuracy: 0.9669 - val_loss: 0.1092 - val_accuracy: 0.9604


In [460]:
predictions = model.predict(X_test)

998/998 [==============================] - 28s 28ms/step


In [481]:
predictions = predictions.reshape(-1)

In [485]:
for i in range(0, predictions.shape[0]):
    if (predictions[i] >= 0.5):
        predictions[i] = 1
    else:
        predictions[i] = 0

In [486]:
from sklearn.metrics import accuracy_score

In [487]:
accuracy_score(predictions, test_df['spam'])

0.9626194579351403

In [515]:
test_s = ["change the sentence to something else, asshole"]

In [539]:
X_test_s = preprocessInputText(test_s, 100)

In [540]:
X_test_s.shape

(1, 100)

In [517]:
test_prediction = model.predict(X_test_s)

1/1 [==============================] - 0s 29ms/step


In [518]:
test_prediction

array([[0.9892102]], dtype=float32)

In [553]:
def getOutputClass(p):
    if p >= 0.5:
        return 1
    else:
        return 0

In [544]:
new_test = ['hello my name is bob, you suck']

In [545]:
pred = model.predict(preprocessInputText(new_test, 100))

1/1 [==============================] - 0s 29ms/step


In [546]:
getOutputClass(pred)

array([[ True]])

In [560]:
def isCommentToxic(comment):
    pred = model.predict(preprocessInputText([comment], 100))
    return getOutputClass(pred)

In [548]:
isCommentToxic('hello my name is bob, you suck')

1/1 [==============================] - 0s 28ms/step


array([[ True]])

In [549]:
isCommentToxic('hello my name is bob')

1/1 [==============================] - 0s 33ms/step


array([[False]])

In [550]:
isCommentToxic('bitch')

1/1 [==============================] - 0s 28ms/step


array([[ True]])

In [555]:
isCommentToxic("you are great")

1/1 [==============================] - 0s 30ms/step


0

In [554]:
isCommentToxic("you are stupid")

1/1 [==============================] - 0s 25ms/step


1

In [561]:
isCommentToxic('hi mom')

1/1 [==============================] - 0s 36ms/step


0

In [562]:
isCommentToxic('piss off')

1/1 [==============================] - 0s 40ms/step


1